# Nighttime Light

In [9]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray
import xarray
from pathlib import Path
import os
from tqdm import tqdm
from multiprocessing import Pool
from geocube.api.core import make_geocube
%matplotlib inline
import time
import workers
%matplotlib inline

In [10]:
geo_city = gpd.read_file("../1.Input/2019行政区划/市.shp")
geo_city["市代码"] = geo_city.市代码.astype(int)

In [11]:
def get_light(fpath):
    # fpath = "D:\Projects\CityGovRelocation\0.DataCleaning\1.Input\Light\PANDA_China_2000.tif"
    lightr = rioxarray.open_rasterio(fpath, masked=True)
    light = lightr.rio.clip(geo_city.geometry.values, geo_city.crs, from_disk=True).sel(band=1).drop("band")
    light.name = "light"

    out_grid = make_geocube(vector_data=geo_city, measurements=["市代码"], like=light,)
    out_grid["light"] = (light.dims, light.values, light.attrs, light.encoding)

    grouped_elevation = out_grid.drop("spatial_ref").groupby(out_grid.市代码)
    grid_mean = grouped_elevation.mean().rename({"light": "light_mean"})
    grid_min = grouped_elevation.min().rename({"light": "light_min"})
    grid_max = grouped_elevation.max().rename({"light": "light_max"})
    grid_std = grouped_elevation.std().rename({"light": "light_std"})

    df_light = xarray.merge([grid_mean, grid_min, grid_max, grid_std]).to_dataframe()
    df_light["file"] = fpath.stem
    return df_light

In [12]:
dpath = "../1.Input/Light"

for (root, dirs, filenames) in os.walk(dpath, topdown=False):
    for _file in filenames:
        print(_file)
        fpath = Path(root).joinpath(_file)
        if fpath.suffix in ['.tif', 'tiff'] and not Path(f"../2.Output/{fpath.stem}_light_city.csv").is_file():
            start = time.time()
            df = get_light(fpath)
            df.to_csv(f"../2.Output/{fpath.stem}_light_city.csv")
            end = time.time()
            print(end - start)

PANDA_China_1984.tif
65.67928981781006
PANDA_China_1985.tif
63.0443651676178
PANDA_China_1986.tif
60.29082655906677
PANDA_China_1987.tif
74.18614625930786
PANDA_China_1988.tif
78.25558400154114
PANDA_China_1989.tif
88.8729841709137
PANDA_China_1990.tif
68.72291421890259
PANDA_China_1991.tif
63.85154628753662
PANDA_China_1992.tif
64.10802388191223
PANDA_China_1993.tif
69.65729284286499
PANDA_China_1994.tif
63.651506423950195
PANDA_China_1995.tif
61.61420249938965
PANDA_China_1996.tif
65.29681658744812
PANDA_China_1997.tif
62.83100628852844
PANDA_China_1998.tif
59.721548557281494
PANDA_China_1999.tif
67.15787839889526
PANDA_China_2000.tif
68.40954256057739
PANDA_China_2001.tif
68.82632040977478
PANDA_China_2002.tif
60.53530836105347
PANDA_China_2003.tif
56.58320951461792
PANDA_China_2004.tif
59.137006521224976
PANDA_China_2005.tif
66.2095091342926
PANDA_China_2006.tif
58.688655376434326
PANDA_China_2007.tif
55.97712683677673
PANDA_China_2008.tif
59.14943265914917
PANDA_China_2009.tif
58.